In [1]:
import os
import sys
import json
import pandas as pd

sys.path.append("../helpers/")
from datasets import *

data_path = "data"
datasets_path = "datasets"

%load_ext autoreload
%autoreload 2

# Datasets
## Pre-processing

In [2]:
RB = pd.read_parquet(os.path.join(data_path, "RB.parquet"))
print_tag_distributions(RB)

food prices      1 items (0.1%) on    1 pages on    1 issues
not a table     22 items (2.7%) on   18 pages on   11 issues
stock          786 items (95.6%) on  470 pages on  310 issues
table           13 items (1.6%) on    9 pages on    9 issues


In [3]:
RB = RB[RB['tag'] != 'not a table']
RB['tag'] = 'table'
print_tag_distributions(RB)

table    800 items (100.0%) on  474 pages on  311 issues


In [4]:
RB.head()

,tag,date_tag,auto_tag,pid,year,month,day,journal,edition,page,meta_issue_id,tb_coords
JDG-1826-02-16-a-p0004-1-0,table,2021-11-11 12:23:16,False,JDG-1826-02-16-a-p0004,1826,2,16,JDG,a,4,JDG-1826-02-16-a,"{'all_points_x': None, 'all_points_y': None, '..."
JDG-1852-11-26-a-p0004-1-0,table,2021-11-11 12:32:00,False,JDG-1852-11-26-a-p0004,1852,11,26,JDG,a,4,JDG-1852-11-26-a,"{'all_points_x': None, 'all_points_y': None, '..."
JDG-1857-05-17-a-p0002-1-0,table,2021-11-11 12:24:16,False,JDG-1857-05-17-a-p0002,1857,5,17,JDG,a,2,JDG-1857-05-17-a,"{'all_points_x': None, 'all_points_y': None, '..."
JDG-1857-05-17-a-p0004-1-0,table,2021-11-11 12:32:10,False,JDG-1857-05-17-a-p0004,1857,5,17,JDG,a,4,JDG-1857-05-17-a,"{'all_points_x': None, 'all_points_y': None, '..."
JDG-1857-06-17-a-p0003-1-0,table,2021-11-11 12:32:11,False,JDG-1857-06-17-a-p0003,1857,6,17,JDG,a,3,JDG-1857-06-17-a,"{'all_points_x': None, 'all_points_y': None, '..."


## Export
### VIA format

In [5]:
via_annotations_RB = dict()    
for i, row in RB.iterrows(): 
    pid = row['pid']
    if pid not in via_annotations_RB:
        via_annotations_RB[pid] = dict()
        via_annotations_RB[pid]['filename'] = pid + '.png'
        via_annotations_RB[pid]['size'] = -1
        via_annotations_RB[pid]['file_attributes'] = {}
        via_annotations_RB[pid]['regions'] = []

    coords = row['tb_coords'].copy()
    if coords['name'] == 'rect':
        del coords['all_points_x']
        del coords['all_points_y']
        coords['x'] = int(coords['x'])
        coords['y'] = int(coords['y'])
        coords['width'] = int(coords['width'])
        coords['height'] = int(coords['height'])
        
    elif coords['name'] == 'polygon':
        coords['all_points_x'] = [int(x) for x in coords['all_points_x']]
        coords['all_points_y'] = [int(y) for y in coords['all_points_y']]
        del coords['x']
        del coords['y']
        del coords['width']
        del coords['height']
    
    region = {}
    region['shape_attributes'] = coords
    region['region_attributes'] = {'label': row['tag']}
    via_annotations_RB[pid]['regions'].append(region)

In [7]:
filename = "via_annotations_RB"

with (open(os.path.join(datasets_path, filename + ".json"), "w")) as f:
    json.dump(via_annotations_RB, f)

In [16]:
filename = "via_annotations_RB"

with (open(os.path.join(datasets_path, filename + ".json"), "r")) as f:
    via_annotations_RB = json.load(f)

### COCO format

In [9]:
with open(os.path.join(data_path, "RB_metadata_images.json"), "r") as f:
    RB_metadata_images = json.load(f)

In [10]:
coco_annotations_RB = convert_via_to_coco(via_annotations_RB, RB_metadata_images)

In [12]:
filename = "coco_annotations_RB"

with (open(os.path.join(datasets_path, filename + ".json"), "w")) as f:
    json.dump(coco_annotations_RB, f)

In [14]:
filename = "coco_annotations_RB"

with (open(os.path.join(datasets_path, filename + ".json"), "r")) as f:
    coco_annotations_RB = json.load(f)